In [46]:
import tensorflow as tf
import numpy as np
import utils 
import numpy.random as npr

In [47]:
class MyCNN:    
    ### 1.___ Initialize the class :
    """
    scope -> name of scope to use for the variables of the model
    data  -> tf.data containing image and label
    summmaries_dir -> directory to store sumaries in case we want to use tensorboard
    """
    def __init__(self, scope='network_CNN', num_classes=10,
                 summaries_dir=None, training = False, keep_prob=1):
        # variables of the model :
        self.scope = scope
        self.keep_prob = keep_prob
        self._training = training
        self.num_classes = num_classes
        # Write tensorboard summaries :
        self.summary_writer = None
        #build model within scope, update summaries if needed :
        with tf.variable_scope(scope):
            #get the graph :
            self.__build_model()
            if summaries_dir :
                # Get the directory for the summaries :
                summary_dir = os.path.join(summaries_dir, "summaries_{}".format(scope))
                # if directory doesnt exist -> create it :
                if not os.path.exists(summary_dir):
                    os.makedirs(summary_dir)
                # Write summaries in summary_writer :
                self.summary_writer = tf.summary.FileWriter(summary_dir)
    
    ### 2.____ Build the graph for the model :  
    def  __build_model(self):         
        #####_____________1. Placeholders for data _____________________________________
        self.img = tf.placeholder(shape=[None, 84, 84, 4], dtype=tf.float32, name="image")
        self.label = tf.placeholder(shape=[10], dtype=tf.float32, name="label")
        
        #####_____________2. Design the graph __________________________________________
        with tf.variable_scope('CONV1', reuse=tf.AUTO_REUSE) as scope:
            conv1 = tf.layers.conv2d(self.img, filters = 24, kernel_size = [3, 3], \
                                padding='SAME', activation=tf.nn.relu,
                                kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv1')
        # Second Convolutional lyer :
        with tf.variable_scope('CONV2', reuse=tf.AUTO_REUSE) as scope:
            conv2 = tf.layers.conv2d(conv1, filters = 12, kernel_size = [3, 3], \
                                padding='SAME', activation=tf.nn.relu,
                                kernel_initializer=tf.contrib.layers.xavier_initializer(), name='conv2')
        # Third a Fully-connected layer :
        with tf.variable_scope('FC1', reuse=tf.AUTO_REUSE) as scope:
            flattened = tf.contrib.layers.flatten(conv2)
            fc1 = tf.contrib.layers.fully_connected(flattened, 512)
        # Dropout :
        with tf.variable_scope('DROPOUT', reuse=tf.AUTO_REUSE) as scope:
            fc1 = tf.layers.dropout(fc1, self.keep_prob, training=self._training, name='Dropout') 
            
        with tf.variable_scope('FC2', reuse=tf.AUTO_REUSE) as scope:
            fc2 = tf.contrib.layers.fully_connected(fc1, 10)
            self.logits_ = fc2
        
        with tf.variable_scope('probabilities', reuse=tf.AUTO_REUSE) as scope:
            self.probabilities = tf.nn.softmax(self.logits_)
        
        #####_____________3. Get the loss and define the optimization operation : ______
        # define the loss :
        with tf.variable_scope('loss', reuse=tf.AUTO_REUSE) as scope:
            self.loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits( \
                                    logits=self.logits_, labels=self.label))
        
        with tf.variable_scope('OPT', reuse=tf.AUTO_REUSE) as scope:
            self.optimizer = tf.train.AdamOptimizer(learning_rate=self._learning_rate)
            self.train = self.optimizer.minimize(self.loss)
                
        # _____________4. Summaries for Tensorboard : __________________________________
        self.summaries = tf.summary.merge([tf.summary.scalar("loss", self.loss)])
        
        
    ### 3.____ Prediction function :
    """
    sess -> tf.Session() to run the graph
    images -> set of images that we want to classify
    """
    def predict(self, images):
        self.keep_prob = 1
        self._training = False 
        with tf.Session() as sess :
            sess.run(tf.global_variables_initializer())
            prediction = sess.run(tf.argmax(self.probabilities, axis=1), {self.img: images})
            return prediction    
    
    """
    __train_step : one training step fct to use for fitting the data
    inputs :
        images, labels = data to train with
    """
    def __train_step(self, sess, images, labels):
        
        self._training = True
        feed_dict = {self.img: images, self.label: labels}
        summaries, _, loss = sess.run([self.summaries, self.train, self.loss], feed_dict)
        if self.summary_writer:
            self.summary_writer.add_summary(summaries)                      
        return loss            

In [48]:
dataset = [npr.randint(0, 200, [100, 84, 84, 4]), npr.randint(0, 10, [100])]
print(len(dataset[0]))

100


In [49]:
def get_next_batch(data, batch_size):
    
    len_data = len(dataset[0])
    perm = npr.permutation(range(len_data))
    indices = perm[:batch_size]
    images = [ dataset[0][i] for i in indices ]
    labels = [ dataset[1][i] for i in indices ]
    return images, labels

In [50]:
def fit(dataset, n_epochs=10, n_batches=100, batch_size=24, 
        save_=True, save_epoch=10, save_directory='checkpoint_directory/MyCNN'):
        
        if save_:
            # create a saver object
            saver = tf.train.Saver()
            
        with tf.Session() as sess :            

            sess.run(tf.global_variables_initializer())
            total_loss = 0
            start_time = time.time()
            
            for i in range(n_epochs):
                
                batch=0 #initiaize batch to zero
                while batch<n_batches :                    
                    images, labels = get_next_batch(dataset, batch_size=batch_size)
                    loss_ = self.__train_step(sess, images, labels)
                    total_loss += loss_
                    batch += 1
                # print total loss for the select epoch :    
                print('Average loss epoch {0}: {1}'.format(i, total_loss/batche))
                
                # save if (epoch+1)%10 = 0
                
                if save_ and (i+1)%save_epoch == 0:
                    saver.save(sess, save_directory, global_step=i)                    
                
            # print the running time :    
            print('Total time: {0} seconds'.format(time.time() - start_time))
   
